In [2]:
import spacy
import pickle
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

### LSTM

In [3]:
model = load_model('../models/intent_lstm_model.keras')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

with open('../models/tokenizer.pkl', 'rb') as file:
    tokenizer = pickle.load(file)
with open('../models/label_encoder.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

def lstm_classify_intent(text):
    text = text.lower()
    
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=model.input_shape[1], padding='post')
    
    prediction = model.predict(padded_sequence)
    
    predicted_index = np.argmax(prediction, axis=1)[0]
    classified_intent = label_encoder.inverse_transform([predicted_index])[0]
    confidence = np.max(prediction)
    
    return classified_intent, confidence

In [17]:
test_text = "Good afternoon! My name is Emma, and my ID is 1234567. The exam session starts next week. What is my registration status for Computer Networks exam?"
classified_intent, confidence = lstm_classify_intent(test_text)
if (confidence >= 0.9):
    print(f"Classified Intent: {classified_intent}, Confidence: {confidence:.3f}")
else:
    print(f"Unknown Intent, Confidence: {confidence:.3f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Classified Intent: query_exam_status, Confidence: 0.997


### NER Model

In [5]:
import string
nlp = spacy.load("../models/custom_ner_model")

test_text = "Good afternoon! My name is Alice, and my ID is 9012343. The exam session starts next week. What is my registration status for Computer Networks exam?"
doc = nlp(test_text)

for ent in doc.ents:
    clean_text = ent.text.strip(string.punctuation)
    print(f"{ent.text}: {ent.label_}")

Alice: last_name
9012343.: matriculation_number
Computer Networks: course_name
